In [1]:
!pip install ultralytics
!pip install numpy
!pip install opencv-python


Prediction
https://docs.ultralytics.com/modes/predict/#inference-arguments

In [1]:
def getResults(img):
  test = processImg(img)
  someJson = predict(test)
  result = readOutput(someJson)
  return result

#getResults(take_photo())
import json
from PIL import Image
from matplotlib import  pyplot as plt
def processImg(path):
  image_path = path
  image_pil = Image.open(image_path)
  image_w, image_h = image_pil.size
  new_image = preprocess_input(image_pil, 640, 640)

  new_image_pil = Image.fromarray((new_image[0] * 255).astype('uint8'))
  #new_image_pil.save("/content/new_img.jpg")

  #plt.imshow(new_image[0])
  #plt.show()

  return new_image_pil

def readOutput(vals):
  jsonVals=json.loads(vals)
  names = []
  confidence=[]
  for item in jsonVals:
    #print(item)
    names.append(item['name'])
    confidence.append(item['confidence'])
  print(names)
  print(confidence)

  if ('openmouth' in names) and ('closedmouth' in names):
    openIndex=names.index('openmouth')
    closedIndex=names.index('closedmouth')
    if confidence[openIndex] > confidence[closedIndex]:
      return 'open'
    else:
      return 'closed'
  elif 'openmouth' in names:
    return 'open'
  else:
    return 'closed'
  
#@title some helper functions

def predict(path):
  results = model(path)
  result = results[0]
  boxes = result.boxes  # Boxes object for bounding box outputs
  masks = result.masks  # Masks object for segmentation masks outputs
  keypoints = result.keypoints  # Keypoints object for pose outputs
  probs = result.probs  # Probs object for classification outputs
  obb = result.obb  # Oriented boxes object for OBB outputs
  #result.show()  # display to screen


  #result.save(filename="result.jpg")  # save to disk
  return result.tojson(normalize=True)
  #return probs
#thing = predict("/content/drive/MyDrive/mewingAI/datasets/FacialFeatures_split/test/images/00a0fd8177a1db74_jpg.rf.e66cca8cc8cccf937b160ea6546b1fbb.jpg")

import numpy as np
import cv2

def preprocess_input(image_pil, net_h=416, net_w=416):
    image = np.asarray(image_pil)
    new_h, new_w, _ = image.shape
    # print("net:", net_h, net_w)
    # print("old:",new_h, new_w)
    # determine the new size of the image
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)/new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)/new_h
        new_h = net_h
    new_w = int(new_w)
    new_h = int(new_h)
    # print("new:",int(new_h), int(new_w))
    # resize the image to the new size
    #resized = cv2.resize(image[:,:,::-1]/255., (int(new_w), int(new_h)))
    resized = cv2.resize(image/255., (int(new_w), int(new_h)))

    # embed the image into the standard letter box
    # print("dims:",int((net_h-new_h)//2), int((net_h+new_h)//2), int((net_w-new_w)//2), int((net_w+new_w)//2))
    new_image = np.ones((net_h, net_w, 3)) * 0.5
    new_image[int((net_h-new_h)//2):int((net_h+new_h)//2), int((net_w-new_w)//2):int((net_w+new_w)//2), :] = resized
    new_image = np.expand_dims(new_image, 0)
    # print(new_image.shape)

    return new_image
  

In [3]:
import cv2
from ultralytics import YOLO

cam = cv2.VideoCapture(0)
cv2.namedWindow("test")

model = YOLO('best.pt')

In [4]:
#while True:
from cv2 import *
import time
import winsound

while True:
    result,image = cam.read()
    cv2.imwrite("image.png", image) 
    results = getResults("image.png")

    if results == 'open':
      print("open")
      winsound.Beep(1000, 100)